In [ ]:
#! pip install keras
#! pip install spacy
#! python -m spacy download en_vectors_web_lg

In [1]:
import spacy
import numpy as np

In [2]:
import pandas as pd
import numpy as np
import pickle

In [3]:
data_folder = '/media/siri/78C6823EC681FD1E/minio/data/dq-data/dl/'
input_folder = '/media/siri/78C6823EC681FD1E/minio/data/dq-data/'

In [4]:
X_train = pickle.load(open(input_folder+'X_train.p', 'rb'))
X_test = pickle.load(open(input_folder+'X_test.p', 'rb'))
y_train = pickle.load(open(input_folder+'y_train.p', 'rb'))
y_test = pickle.load(open(input_folder+'y_test.p', 'rb'))

In [5]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 270872 entries, 186150 to 121959
Data columns (total 4 columns):
qid1         270872 non-null int64
qid2         270872 non-null int64
question1    270872 non-null object
question2    270872 non-null object
dtypes: int64(2), object(2)
memory usage: 10.3+ MB


In [ ]:
nlp = spacy.load('en_vectors_web_lg')

In [ ]:
def create_dataset(nlp, X, num_oov, max_length, norm_vectors = True):
    len_q1 = X['question1'].size
    sents = pd.concat([X['question1'], X['question2']]).values
    
    # the extra +1 is for a zero vector represting NULL for padding
    num_vectors = max(lex.rank for lex in nlp.vocab) + 2 
    
    # create random vectors for OOV tokens
    oov = np.random.normal(size=(num_oov, nlp.vocab.vectors_length))
    oov = oov / oov.sum(axis=1, keepdims=True)
    
    vectors = np.zeros((num_vectors + num_oov, nlp.vocab.vectors_length), dtype='float32')
    vectors[num_vectors:, ] = oov
    for lex in nlp.vocab:
        if lex.has_vector and lex.vector_norm > 0:
            vectors[lex.rank + 1] = lex.vector / lex.vector_norm if norm_vectors == True else lex.vector
            
    sents_as_ids = []
    for sent in sents:
        doc = nlp(sent)
        word_ids = []
        
        for i, token in enumerate(doc):
            # skip odd spaces from tokenizer
            if token.has_vector and token.vector_norm == 0:
                continue
                
            if i > max_length:
                break
                
            if token.has_vector:
                word_ids.append(token.rank + 1)
            else:
                # if we don't have a vector, pick an OOV entry
                word_ids.append(token.rank % num_oov + num_vectors) 
                
        # there must be a simpler way of generating padded arrays from lists...
        word_id_vec = np.zeros((max_length), dtype='int')
        clipped_len = min(max_length, len(word_ids))
        word_id_vec[:clipped_len] = word_ids[:clipped_len]
        sents_as_ids.append(word_id_vec)        
        
    return vectors, np.array(sents_as_ids[:len_q1]), np.array(sents_as_ids[len_q1:])

In [ ]:
w2v, q1_train_w2v, q2_train_w2v = create_dataset(nlp, X_train, 100, 50, True)

In [ ]:
_, q1_test_w2v, q2_test_w2v = create_dataset(nlp, X_test, 100, 50, True)

In [ ]:
q1_train_w2v.shape

In [ ]:
pickle.dump(w2v, open(data_folder+'w2v.p', 'wb'))
pickle.dump(q1_train_w2v, open(data_folder+'q1_train_w2v.p', 'wb'))
pickle.dump(q2_train_w2v, open(data_folder+'q2_train_w2v.p', 'wb'))
pickle.dump(q1_test_w2v, open(data_folder+'q1_test_w2v.p', 'wb'))
pickle.dump(q2_test_w2v, open(data_folder+'q2_test_w2v.p', 'wb'))

## Modeling

In [6]:
w2v = pickle.load(open(data_folder+'w2v.p', 'rb'))
q1_train_w2v = pickle.load(open(data_folder+'q1_train_w2v.p', 'rb'))
q2_train_w2v = pickle.load(open(data_folder+'q2_train_w2v.p', 'rb'))
q1_test_w2v = pickle.load(open(data_folder+'q1_test_w2v.p', 'rb'))
q2_test_w2v = pickle.load(open(data_folder+'q2_test_w2v.p', 'rb'))

In [7]:
from keras import layers, Model, models, initializers
from keras.engine.topology import Layer
from keras import backend as K
from keras.optimizers import Adam,RMSprop, Nadam

Using TensorFlow backend.


In [8]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

In [9]:
def create_embedding(vectors, max_length, projected_dim):
    return models.Sequential([
        layers.Embedding(
            vectors.shape[0],
            vectors.shape[1],
            input_length=max_length,
            weights=[vectors],
            trainable=False),
        
        layers.TimeDistributed(
            layers.Dense(projected_dim,
                         activation=None,
                         use_bias=False))
    ])

In [10]:
def create_feedforward(num_units=200, activation='relu', dropout_rate=0.2):
    return models.Sequential([
        layers.Dense(num_units, activation=activation, use_bias=True),
        layers.Dropout(dropout_rate),
        layers.Dense(num_units, activation=activation, use_bias=True),
        layers.Dropout(dropout_rate)
    ])

In [11]:
def create_biLSTM(num_units=200):
    return models.Sequential([
        layers.Bidirectional(layers.LSTM(num_units, return_sequences=True)),
        layers.TimeDistributed(layers.Dense(1, activation='sigmoid'))
    ])

In [12]:
def normalizer(axis):
    def _normalize(att_weights):
        #if biased:
        #    d_shape = K.shape(att_weights)
        #    d_0 = K.repeat_elements(K.reshape(K.arange(0, d_shape[0]), (-1,1)), d_shape[1], axis=1)
        #    d_1 = K.repeat_elements(K.reshape(K.arange(0, d_shape[1]), (1,-1)), d_shape[0], axis=0)
        #    dist = K.clip(K.abs(d_0-d_1), 0, 11)
        #    exp_weights = K.exp(att_weights+dist)
        #else:
        exp_weights = K.exp(att_weights)
        sum_weights = K.sum(exp_weights, axis=axis, keepdims=True)
        return exp_weights/sum_weights
    return _normalize

def sum_word(x):
    return K.sum(x, axis=1)

In [14]:
def build_model(vectors, max_length, num_hidden, num_classes, 
                projected_dim, entail_dir='both', 
                dropout_rate1=0.2, dropout_rate2=0.2, dropout_rate3=0.2):
    input1 = layers.Input(shape=(max_length,), dtype='int32', name='words1')
    input2 = layers.Input(shape=(max_length,), dtype='int32', name='words2')
    
    # embeddings (projected)
    embed = create_embedding(vectors, max_length, projected_dim)
    #norm_input1 = layers.BatchNormalization()(input1)   
    #norm_input2 = layers.BatchNormalization()(input2)   
    a = embed(input1)
    b = embed(input2)
    #a = layers.BatchNormalization()(a)   
    #b = layers.BatchNormalization()(b)   
    
    # step 1: attend
    F = create_feedforward(num_hidden, dropout_rate=dropout_rate1)
    att_weights = layers.dot([F(a), F(b)], axes=-1, normalize=True)
    #att_weights = layers.Lambda(cosine_distance())([F(a), F(b)])
    
    G = create_feedforward(num_hidden, dropout_rate=dropout_rate2)
    
    if entail_dir == 'both':
        norm_weights_a = layers.Lambda(normalizer(1))(att_weights)
        norm_weights_b = layers.Lambda(normalizer(2))(att_weights)
        alpha = layers.dot([norm_weights_a, a], axes=1)
        beta  = layers.dot([norm_weights_b, b], axes=1)
        #alpha = layers.Lambda(cosine_distance())([norm_weights_a, a])
        #beta = layers.Lambda(cosine_distance())([norm_weights_b, b])

        # step 2: compare
        comp1 = layers.concatenate([a, beta])
        comp2 = layers.concatenate([b, alpha])
        v1 = layers.TimeDistributed(G)(comp1)
        v2 = layers.TimeDistributed(G)(comp2)
        v1 = layers.BatchNormalization()(v1)
        v2 = layers.BatchNormalization()(v2)

        # step 3: aggregate
        v1_sum = layers.Lambda(sum_word)(v1)
        v2_sum = layers.Lambda(sum_word)(v2)
        concat = layers.concatenate([v1_sum, v2_sum])
    elif entail_dir == 'left':
        norm_weights_a = layers.Lambda(normalizer(1))(att_weights)
        alpha = layers.dot([norm_weights_a, a], axes=1)
        comp2 = layers.concatenate([b, alpha])
        v2 = layers.TimeDistributed(G)(comp2)
        v2_sum = layers.Lambda(sum_word)(v2)
        concat = v2_sum
    else:
        norm_weights_b = layers.Lambda(normalizer(2))(att_weights)
        beta  = layers.dot([norm_weights_b, b], axes=1)
        comp1 = layers.concatenate([a, beta])
        v1 = layers.TimeDistributed(G)(comp1)
        v1_sum = layers.Lambda(sum_word)(v1)
        concat = v1_sum
    
    
    H = create_feedforward(num_hidden, dropout_rate=dropout_rate3)
    out = H(concat)
    out = layers.Dense(num_classes, activation='sigmoid', use_bias=True)(out)
    
    model = Model([input1, input2], out)
    
    model.compile(optimizer=Nadam(lr=0.00001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [15]:
K.clear_session()
m = build_model(w2v, 50, 200, 1, 200, dropout_rate1=0.2, dropout_rate2=0.2, dropout_rate3=0.2)
m.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
words1 (InputLayer)             (None, 50)           0                                            
__________________________________________________________________________________________________
words2 (InputLayer)             (None, 50)           0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 50, 200)      321381600   words1[0][0]                     
                                                                 words2[0][0]                     
__________________________________________________________________________________________________
sequential_2 (Sequential)       (None, 50, 200)      80400       sequential_1[1][0]               
          

In [16]:
m.fit([q1_train_w2v, q2_train_w2v], y_train, batch_size=150, epochs=200,
      validation_data=([q1_test_w2v, q2_test_w2v], y_test))

Train on 270872 samples, validate on 133415 samples
Epoch 1/200
270872/270872 [==============================] - 44s 162us/step - loss: 0.8595 - acc: 0.6038 - val_loss: 0.6033 - val_acc: 0.6536
Epoch 2/200
270872/270872 [==============================] - 43s 158us/step - loss: 0.6435 - acc: 0.6456 - val_loss: 0.5816 - val_acc: 0.6794
Epoch 3/200
270872/270872 [==============================] - 43s 158us/step - loss: 0.5981 - acc: 0.6744 - val_loss: 0.5672 - val_acc: 0.6982
Epoch 4/200
270872/270872 [==============================] - 43s 158us/step - loss: 0.5737 - acc: 0.6935 - val_loss: 0.5554 - val_acc: 0.7083
Epoch 5/200
270872/270872 [==============================] - 43s 158us/step - loss: 0.5600 - acc: 0.7052 - val_loss: 0.5512 - val_acc: 0.7132
Epoch 6/200
270872/270872 [==============================] - 43s 159us/step - loss: 0.5489 - acc: 0.7146 - val_loss: 0.5364 - val_acc: 0.7265
Epoch 7/200
270872/270872 [==============================] - 43s 159us/step - loss: 0.5397 - acc

Epoch 58/200
270872/270872 [==============================] - 43s 159us/step - loss: 0.4019 - acc: 0.8065 - val_loss: 0.4316 - val_acc: 0.7891
Epoch 59/200
270872/270872 [==============================] - 43s 158us/step - loss: 0.4005 - acc: 0.8084 - val_loss: 0.4324 - val_acc: 0.7883
Epoch 60/200
270872/270872 [==============================] - 43s 159us/step - loss: 0.3991 - acc: 0.8085 - val_loss: 0.4342 - val_acc: 0.7868
Epoch 61/200
270872/270872 [==============================] - 43s 159us/step - loss: 0.3978 - acc: 0.8101 - val_loss: 0.4431 - val_acc: 0.7803
Epoch 62/200
270872/270872 [==============================] - 43s 159us/step - loss: 0.3968 - acc: 0.8103 - val_loss: 0.4407 - val_acc: 0.7821
Epoch 63/200
270872/270872 [==============================] - 43s 159us/step - loss: 0.3957 - acc: 0.8111 - val_loss: 0.4257 - val_acc: 0.7936
Epoch 64/200
270872/270872 [==============================] - 43s 159us/step - loss: 0.3944 - acc: 0.8117 - val_loss: 0.4268 - val_acc: 0.7923

270872/270872 [==============================] - 43s 159us/step - loss: 0.3451 - acc: 0.8397 - val_loss: 0.4112 - val_acc: 0.8052
Epoch 116/200
270872/270872 [==============================] - 43s 159us/step - loss: 0.3442 - acc: 0.8407 - val_loss: 0.4103 - val_acc: 0.8053
Epoch 117/200
270872/270872 [==============================] - 43s 158us/step - loss: 0.3431 - acc: 0.8410 - val_loss: 0.4234 - val_acc: 0.7996
Epoch 118/200
270872/270872 [==============================] - 43s 159us/step - loss: 0.3421 - acc: 0.8419 - val_loss: 0.4157 - val_acc: 0.8026
Epoch 119/200
270872/270872 [==============================] - 43s 158us/step - loss: 0.3417 - acc: 0.8421 - val_loss: 0.4258 - val_acc: 0.7975
Epoch 120/200
270872/270872 [==============================] - 43s 159us/step - loss: 0.3416 - acc: 0.8422 - val_loss: 0.4184 - val_acc: 0.8019
Epoch 121/200
270872/270872 [==============================] - 43s 158us/step - loss: 0.3395 - acc: 0.8427 - val_loss: 0.4138 - val_acc: 0.8055
Epoch 

Epoch 172/200
270872/270872 [==============================] - 43s 158us/step - loss: 0.3064 - acc: 0.8620 - val_loss: 0.4163 - val_acc: 0.8097
Epoch 173/200
270872/270872 [==============================] - 43s 158us/step - loss: 0.3056 - acc: 0.8619 - val_loss: 0.4177 - val_acc: 0.8086
Epoch 174/200
270872/270872 [==============================] - 43s 159us/step - loss: 0.3052 - acc: 0.8620 - val_loss: 0.4081 - val_acc: 0.8117
Epoch 175/200
270872/270872 [==============================] - 43s 159us/step - loss: 0.3041 - acc: 0.8620 - val_loss: 0.4124 - val_acc: 0.8117
Epoch 176/200
270872/270872 [==============================] - 43s 158us/step - loss: 0.3033 - acc: 0.8627 - val_loss: 0.4138 - val_acc: 0.8107
Epoch 177/200
270872/270872 [==============================] - 43s 159us/step - loss: 0.3033 - acc: 0.8629 - val_loss: 0.4118 - val_acc: 0.8110
Epoch 178/200
270872/270872 [==============================] - 43s 159us/step - loss: 0.3025 - acc: 0.8638 - val_loss: 0.4123 - val_acc:

In [18]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

In [56]:
target_names = ['not duplicate', 'duplicate']

In [58]:
m.evaluate([q1_test_w2v, q2_test_w2v],y_test)

133415/133415 [==============================] - 8s 60us/step


[0.41521005990359305, 0.8125323239524732]

In [37]:
y_pred_dl = m.predict([q1_test_w2v, q2_test_w2v], batch_size=150)

In [28]:
convert_binary = lambda x: 1 if x[0] >= .5 else 0

In [53]:
y_pred_dl_classes = np.array([convert_binary(y) for y in y_pred_dl])

In [55]:
score_dl = accuracy_score(y_test.values, y_pred_dl_classes)
rscore_dl = recall_score(y_test.values, y_pred_dl_classes)
pscore_dl = precision_score(y_test.values, y_pred_dl_classes)
print('Accuracy score for DL ', score_dl)
print('Recall score for DL ', rscore_dl)
print('Precision score for DL ', pscore_dl)

Accuracy score for DL  0.8125323239515797
Recall score for DL  0.7615772767966144
Precision score for DL  0.7379295388679692


In [57]:
print(classification_report(y_test.values, y_pred_dl_classes, target_names=target_names))

               precision    recall  f1-score   support

not duplicate       0.86      0.84      0.85     84267
    duplicate       0.74      0.76      0.75     49148

    micro avg       0.81      0.81      0.81    133415
    macro avg       0.80      0.80      0.80    133415
 weighted avg       0.81      0.81      0.81    133415

